In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [2]:
# load the trained model, scaler, and encoder
model = load_model('model.h5')

# load the encoder and scaler
with open('ohe_encoder.pkl', 'rb') as file:
    ohe_encoder = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)        

In [8]:
input_data = {
    'CreditScore': 720,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,  
    'Tenure': 5,
    'Balance': 60000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}

In [6]:
geo_encoded =ohe_encoder.transform([[input_data['Geography']]])

c:\Users\acer\OneDrive\Desktop\ANN project Implementation\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [7]:
geo_encoded


array([[1., 0., 0.]])

In [10]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns = ohe_encoder.get_feature_names_out(['Geography']))

In [11]:
input_df = pd.DataFrame([input_data])

In [12]:
# encode categorical features
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,1,35,5,60000.0,2,1,1,50000.0


In [13]:
# concatenate the encoded geography columns and drop the original Geography column
input_df = pd.concat([input_df.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

In [14]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,720,1,35,5,60000.0,2,1,1,50000.0,1.0,0.0,0.0


In [15]:
# scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 0.70934256,  0.91324755, -0.37056859, -0.00134472, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# prdict Churn data
prediction  = model.predict(input_scaled) 
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step


array([[0.01244732]], dtype=float32)

In [17]:
prediction_prob = prediction[0][0]
if prediction_prob >= 0.5:
    print(f'The customer is likely to churn with a probability of {prediction_prob:.2f}')
else:
    print(f'The customer is unlikely to churn with a probability of {prediction_prob:.2f}')

The customer is unlikely to churn with a probability of 0.01
